Check all hdf5 groups and attributes with:

h5dump -n 1 lalsuitetest.hdf5

This prints the description attribute:

h5dump -a description lalsuitetest.hdf5 

In [ ]:
%pylab inline

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

np.set_printoptions(precision=6, linewidth=110)

In [48]:
import h5py

In [2]:
sys.path.insert(0, '../../src')

import waveform as wave
import waveformset as ws
import trainingset as train
import taylorf2 as f2
import gaussianprocessregression as gpr
import designofexperiment as doe
import lalwaveform
import plotparams
import greedy
import empiricalinterpolation as eim
import surrogate
import diagnostics
import uncertaintysampling as us

import imp
imp.reload(wave)
imp.reload(ws)
imp.reload(train)
imp.reload(f2)
imp.reload(gpr)
imp.reload(doe)
imp.reload(lalwaveform)
imp.reload(greedy)
imp.reload(eim)
imp.reload(surrogate)
imp.reload(diagnostics)
imp.reload(us)

import constants
imp.reload(constants)
from constants import *

# Construct surrogate in way that can be directly converted to lalsuite code

In [42]:
def kernel(x1, x2, hyperparams):
    """Matern covariance function for n-dimensional data.
    
    Parameters
    ----------
    x1 : array with shape ndim
    x2 : array with shape ndim
    hyperparams : array with shape ndim+2 [sigma_f, ls0, ls1, ..., sigma_n]
        sigma_f : Approximately the range (ymax-ymin) of values that the data takes.
            sigma_f^2 called the signal variance.
        sigma_n : Noise term. The uncertainty in the y values of the data.
        lsi : Length scales for the variation in dimension i.
    
    Returns
    -------
    covariance : float
    """
    sigma_f = hyperparams[0]
    sigma_n = hyperparams[-1]
    ls = hyperparams[1:-1]
    ndim = len(ls)
    
    # Noise nugget for diagonal elements
    if np.array_equal(x1, x2):
        nugget = sigma_n**2
    else:
        nugget = 0.0
    
    # r**2
    rsq = np.sum(np.array([(x1[i]-x2[i])**2 / ls[i]**2 for i in range(ndim)]))
    r = np.sqrt(rsq)
    
    # nu = 5/2 Matern covariance
    matern = (1. + np.sqrt(5.)*r + 5.*r**2/3.) * np.exp(-np.sqrt(5.)*r)
    
    # Full covariance
    # You must include the nugget to agree with scikit-learn when the points x1, x2 are exactly the same
    return sigma_f**2 * matern + nugget

In [43]:
def gp_predict(xst, hyperparams, x_train, Kinv_dot_y):
    """Interpolate the function at the point xst using Gaussian process regression.
    
    Parameters
    ----------
    xst : array of shape ndim.
        Point x_* where you want to evaluate the function.
    hyperparams : array with shape ndim+2 [sigma_f, ls0, ls1, ..., sigma_n].
        Hyperparameters for the GPR kernel.
    x_train : array of shape (n_train, ndim).
        Training set points.
    Kinv_dot_y : array of shape n_train.
        The interpolating weights at each training set point.
    
    Returns
    -------
    yst : float
        Interpolated value at the point xst.
    """
    # Evaluate vector K_*
    Kst = np.array([kernel(xst, x, hyperparams) for x in x_train])

    # Evaluate y_*
    return np.dot(Kst, Kinv_dot_y)

In [44]:
def extract_data_from_scikit_learn(gp):
    """Extract the data in the scikit-learn GaussianProcessRegressor class 
    that you need for the lalsuite version.
    """
    # hyperparams = np.array([sigma_f, lq, ls1, ls2, llam1, llam2, sigma_n])
    hyperparams = gpr.get_hyperparameters(gp)
    
    # The training data
    x_train = gp.X_train_
    y_train = gp.y_train_
    
    # Evaluate K
    K = np.array([[kernel(x1, x2, hyperparams) for x2 in x_train] for x1 in x_train])
    
    # Evaluate K^{-1}
    Kinv = np.linalg.inv(K)
    
    # Evaluate (K^{-1})_{ij} y_j (array of length nparams).
    Kinv_dot_y = np.dot(Kinv, y_train)
    
    return hyperparams, x_train, Kinv_dot_y

# Load scikit-learn (python) version of surrogate

In [45]:
Bamp_filename = '../../data/teobtest40hz/B_amp_corners_lhd.hdf5'
Bphase_filename = '../../data/teobtest40hz/B_phase_corners_lhd.hdf5'
gp_amp_filename = '../../data/teobtest40hz/gp_amp_corners_lhd.hdf5'
gp_phase_filename = '../../data/teobtest40hz/gp_phase_corners_lhd.hdf5'
sur = surrogate.GPSurrogate.load(Bamp_filename, Bphase_filename, gp_amp_filename, gp_phase_filename)

# Bamp_filename = '../../data/teobtest40hz/B_amp_lhd_uncsamp.hdf5'
# Bphase_filename = '../../data/teobtest40hz/B_phase_lhd_uncsamp.hdf5'
# gp_amp_filename = '../../data/teobtest40hz/gp_amp_lhd_uncsamp.hdf5'
# gp_phase_filename = '../../data/teobtest40hz/gp_phase_lhd_uncsamp.hdf5'
# sur = surrogate.GPSurrogate.load(Bamp_filename, Bphase_filename, gp_amp_filename, gp_phase_filename)

In [46]:
#Random point:
#x = np.array([0.8, 0.2, 0.1, 1000, 2000])

# Point exactly in training set:
x = np.array([3.333333e-01, -4.000000e-01, -4.000000e-01, 1.000000e-01, 1.000000e-01])

for i in range(len(sur.dphase_gp_list)):
    gp = sur.dphase_gp_list[i]
    #gp = sur.dphase_gp_list[0]
    hyperparams, x_train, Kinv_dot_y = extract_data_from_scikit_learn(gp)

    a = gp.predict(np.atleast_2d(x))[0]
    b = gp_predict(x, hyperparams, x_train, Kinv_dot_y)

    #sigma_n = hyperparams[-1]
    #print sigma_n
    print np.abs(b/a-1.)

5.53723478181e-09
6.42491049252e-09
1.96401661601e-09
5.68317459759e-09
4.66294558521e-09
2.28728636031e-09
2.02868855048e-09
2.36434094525e-10
1.48636081221e-09
2.49062059865e-09
4.93199625584e-09
1.76905234905e-09
3.07067204997e-09
1.08489428552e-09
4.44460579452e-10
2.02839578467e-09
3.19912274449e-09
3.97346355818e-09
1.37696076763e-09
3.79370002079e-09


# Generate hdf5 file for lalsuite version

In [139]:
def lalsuite_surrogate_format(filename, sur):
    """Write data to an hdf5 file format that can be read by the 
    lalsuite version of the code.
    """
    f = h5py.File(filename, libver='latest')
    
    namp = len(sur.Bamp)
    nphase = len(sur.Bphase)
    
    f.attrs['description'] = \
'''
********************************************************************************
Data for TEOBv4_ROM reduced order model (aligned-spin BNS with tidal interactions).

See B. Lackey, et al. arXiv:xxxx.xxxx.

Parameter ranges:
* 1/3 <= q <= 1
* -0.4 <= spin_1z <= 0.4
* -0.4 <= spin_2z <= 0.4
* 0.1 <= lambda_1 <= 3000
* 0.1 <= lambda_2 <= 3000
* flow >= xxHz

This ROM was built using the TEOBv4 waveform.

The hyperparameters for the Gaussian process regression associated with each 
basis function are listed in the order
[sigma_f, l_q, l_spin1z, l_spin2z, l_lambda1, l_lambda2, sigma_n]
where sigma_f is approximately the function range, sigma_n is the noise/tollerance, 
and l_i is the correlation length scale for the parameter i.
********************************************************************************
'''
    
    # Frequency samples
    f['mf'] = sur.mf
    
    # Training set samples.
    # They are the same for all basis functions so pick amp_0
    gp = sur.damp_gp_list[0]
    x_train = gp.X_train_
    f['x_train'] = x_train
    
    print 'Writing amplitude bases...'
    for i in range(namp):
        print i,
        groupname = 'delta_ln_a_' + str(i)
        group = f.create_group(groupname)
        
        group.attrs['mf_node'] = sur.mf_amp[i]
        group['basis'] = sur.Bamp[i].amp
    
        gp = sur.damp_gp_list[i]
        hyperparameters, x_train, kinv_dot_y = extract_data_from_scikit_learn(gp)
        group['hyperparameters'] = hyperparameters
        group['kinv_dot_y'] = kinv_dot_y
        
    print '\nWriting phase bases...'
    for i in range(nphase):
        print i,
        groupname = 'delta_phi_' + str(i)
        group = f.create_group(groupname)
        
        group.attrs['mf_node'] = sur.mf_phase[i]
        group['basis'] = sur.Bphase[i].phase
        
        gp = sur.dphase_gp_list[i]
        hyperparameters, x_train, kinv_dot_y = extract_data_from_scikit_learn(gp)
        group['hyperparameters'] = hyperparameters
        group['kinv_dot_y'] = kinv_dot_y
        
    f.close()

In [140]:
filename = 'lalsuitetest.hdf5'
lalsuite_surrogate_format(filename, sur)

Writing amplitude bases...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
Writing phase bases...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19


## Testing

In [141]:
f = h5py.File(filename, libver='latest')

In [142]:
print f['mf'][:].shape
print f['x_train'][:].shape

(10000,)
(159, 5)


In [143]:
print f['delta_ln_a_19'].attrs['mf_node']
print f['delta_ln_a_19/hyperparameters'][:]
print f['delta_ln_a_19/kinv_dot_y'][:]
print f['delta_ln_a_19/basis'][:]

0.03
[  1.244054e+00   5.415526e-01   2.404507e+00   3.200000e+00   1.210941e+03   6.054942e+03   1.816954e-02]
[  1.353636e-01  -2.426995e+00   1.238622e+00   2.569431e+00   3.361933e+00   1.419202e+01  -3.627787e+00
  -8.324047e-01  -1.272532e-01  -3.368970e+01  -2.408338e+00  -1.902484e+00   5.119303e+00   4.062834e+00
   1.460107e+00   3.370889e-01  -2.491526e+00   5.354783e+00   7.465732e-01   4.420478e+01   3.176610e+00
  -5.925059e+00   1.808938e+00  -1.110560e+01   5.000668e+00   6.217673e+00   4.195432e+00  -1.312605e+01
   2.338496e-01   5.919319e+00  -5.379469e+00   1.755869e+01  -1.626893e+01   1.689817e+01  -1.431328e+01
  -1.971245e+01   4.480975e+00   3.670967e+00  -8.873717e+00  -3.385341e+00   6.651964e+00  -8.662202e+00
   8.775134e+00   1.007829e+01   6.649881e+00   3.344075e+00  -4.441547e+00   4.264576e+00  -4.622621e+01
   2.691734e+00  -5.929247e+00   1.690460e+01  -3.125132e+00  -6.299917e+00  -6.306608e+00   1.515829e+00
  -2.227001e+00   7.702309e+00  -3.24518

In [144]:
print f['delta_phi_19'].attrs['mf_node']
print f['delta_phi_19/hyperparameters'][:]
print f['delta_phi_19/kinv_dot_y'][:]
print f['delta_phi_19/basis'][:]

0.03
[  4.255004e+02   1.421491e+00   8.000000e+00   8.000000e+00   8.187851e+03   2.999900e+04   5.062750e-04]
[ 0.161461  0.096188  0.054938  0.327947  0.020727  0.155349 -0.008214 -0.051078  0.235633 -1.000358 -0.042436
 -0.307369  0.988341  0.073701 -0.252881 -0.108205  0.246872  0.052102  0.069441  0.30316  -0.022522 -0.11242
  0.059793 -0.139292  0.20552   0.93233   0.06626  -0.077352  0.774753 -0.627922  0.019855 -0.027874 -0.483242
  0.005088 -0.785822  0.01263   0.400281 -0.800456 -0.580188 -0.260387  0.319411 -0.537523 -0.132596  1.483104
  0.070138 -0.224954 -0.858842  0.713094 -0.410665  0.833558 -0.047864  1.206982 -0.867527 -0.117764  0.446977
  0.055848 -0.876076  0.164289 -0.036691 -0.899262  0.083572  0.165041  0.863735  0.356303 -1.582601  2.375573
  0.56885  -0.553988  0.031443  0.069171  0.268207 -0.297621 -1.257951  0.369415 -0.753824 -0.027229 -0.904814
  0.22315  -0.271993  0.689553  0.709485 -0.88239  -0.703084 -0.80746  -0.595217  0.274676 -0.718739 -0.869145
 

In [145]:
f.close()